In [1]:
import librosa
import math
import matplotlib.pyplot as plt
import numpy as np

### Reading .wav files 

In [2]:
src, sr1 = librosa.load('trs.wav', sr = None)
noise, sr2 = librosa.load('trn.wav', sr = None)
x_nmf, sr3 = librosa.load('x_nmf.wav', sr = None)

### STFT Function

In [3]:
def STFT(x,N,hop) :
    F = np.zeros((N,N),dtype=np.complex_)
    for i in range(N):
        for j in range(N):
            F[i,j] = np.exp(-1j*2*math.pi*i*j/N )
    frames = (len(x)-hop)/hop
    X = np.zeros((N,frames))
    for i in range(frames):
        X[:,i] = x[i*hop:i*hop+N] * np.hanning(N)    
    Y = np.matmul(F,X)
    return Y    

### ISTFT Function

In [4]:
def ISTFT(X,N,hop,len_signal):
    frames = X.shape[1]
    inverse_F = np.zeros((N,N),dtype=np.complex_)
    for i in range(N):
        for j in range(N):
            inverse_F[i,j] = np.exp(1j*2*math.pi*i*j/N )/N
    X_inv = np.matmul(inverse_F,X)
    X_inv = np.real(X_inv)
    x_inv = np.zeros(shape=(frames, len_signal),dtype=np.complex_)
    for i in range(frames):
        x_inv[i, (hop * i):(hop* i) + 1024] = X_inv[:, i]
    x_inv = x_inv.sum(axis=0)
    return x_inv.real

### NMF Function

In [5]:
def NMF(X,L):
    W = np.asmatrix(np.random.rand(513,L))
    H = np.asmatrix(np.random.rand(L,987))
    I = np.asmatrix(np.ones((X.shape[0],X.shape[1])))
    for i in range(2000):
        W = np.multiply(W,(X/(W*H+10**-14))*(H.T)/(I*H.T+10**-14))     
        H = np.multiply(H, (W.T*(X/(W*H+10**-14)))/(W.T*I+10**-14))
    return (W,H)

### Getting magnitude matrix of STFT

In [6]:
src_stft = STFT(src, 1024, 512)
src_stft_half = src_stft[0:513,:]
src_stft_real = abs(src_stft_half)
src_stft_real.shape

(513, 987)

In [7]:
noise_stft = STFT(noise, 1024, 512)
noise_stft_half = noise_stft[0:513,:]
noise_stft_real = abs(noise_stft_half)
noise_stft_real.shape

(513, 987)

In [8]:
xnmf_stft = STFT(x_nmf, 1024, 512)
xnmf_stft_half = xnmf_stft[0:513,:]
xnmf_stft_real = abs(xnmf_stft_half)
xnmf_stft_real.shape

(513, 129)

### NMF approximation

In [9]:
W_s,H_s = NMF(src_stft_real,30)
W_n,H_n = NMF(noise_stft_real,30)

In [10]:
W_nmf = np.asmatrix(np.concatenate([W_s,W_n], axis = 1))
H_nmf = np.asmatrix(np.random.rand(60,129))
I = np.asmatrix(np.ones((xnmf_stft_real.shape[0],xnmf_stft_real.shape[1])))
for i in range(2000):
    H_nmf = np.multiply(H_nmf, (W_nmf.T*(xnmf_stft_real/(W_nmf*H_nmf+10**-14)))/(W_nmf.T*I+10**-14))

In [11]:
M = (W_s * H_nmf[0:30,:])/(W_nmf*H_nmf + 10**-14)

In [13]:
S_hat = np.multiply(M,xnmf_stft_half)

In [18]:
S_hat_full = np.zeros((1024,129), dtype=np.complex_)
for i in range(0,513):
    S_hat_full[i,:] = S_hat[i,:]
for i in range(513,1024):
    S_hat_full[i,:] = np.conj(S_hat[511-(i-513),:])

In [20]:
x_clean = ISTFT(S_hat_full,1024,512, len(x_nmf))

In [21]:
librosa.output.write_wav('x_clean.wav', x_clean, sr3)